In [1]:
from glob import glob
import numpy as np

from sklearn.model_selection import train_test_split

from torch.utils.data import DataLoader

import torch
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable

from Infrastructure import CaptchaDataset, CNN

import logging
logging.basicConfig(level=logging.DEBUG)

In [2]:
CUDA = False
if torch.cuda.is_available():
    CUDA = True
print(f'CUDA available: {CUDA}')

CUDA available: False


In [3]:
file_locations = glob('./Captchas/*')
captcha_names = [file.split('/')[-1].split('.')[0] for file in file_locations]
print( f'identified {len(file_locations)} images' )

identified 113062 images


In [4]:
# Unique characters is global -- expect only 60 out
unique_characters = [*set(char for name in captcha_names for char in name)]

print( f'{len(unique_characters)} unique characters: {unique_characters}' )

60 unique characters: ['m', 'U', 'Y', '4', 'z', '8', '5', 'u', 'k', 'X', 'N', 'b', 'J', '3', 'B', 'j', 'y', 'T', 'l', 'a', 'Q', 'C', 'Z', 'H', 'P', '7', '6', 'L', 'e', 'v', 'h', 'R', 'n', 'O', 'd', 'S', 'p', 'i', 'w', 'f', 'D', 'r', 'g', 'A', 'G', 'V', '2', 't', 'F', 's', 'x', 'q', 'M', '9', 'K', 'E', 'I', 'c', '1', 'W']


In [5]:
# Split training/test data
train_files, test_files = train_test_split(file_locations[0:3_000], test_size = .2)
print(f'Split dataset into 80:20 train/test of sizes {len(train_files)},{len(test_files)}.')

Split dataset into 80:20 train/test of sizes 2400,600.


In [6]:
# Load in training files.
train = CaptchaDataset.from_dir(train_files)#[0:128])

print(f'{train.X.shape}\nSample of images in format 40px x 150px x 3 RGB channels, of type {type(train.X[0][0][0][0])}')

# Instantiate dataloader (the iterable that provides batches for gradient descent.)
dl = DataLoader(train, \
    64, # Fetch 4 samples per batch
    shuffle=True, num_workers=2)

(2400, 40, 150, 3)
Sample of images in format 40px x 150px x 3 RGB channels, of type <class 'numpy.float32'>


In [7]:
# Test out the iterable.
dataiter = iter(dl)
images, label_array, labels = next(dataiter)
print(f'Each batch has a dataset of shape {images.shape} and a corresponding set of {label_array.shape} labels.')

Each batch has a dataset of shape torch.Size([64, 3, 40, 150]) and a corresponding set of torch.Size([64, 5, 62]) labels.


In [9]:
net = CNN()

net.fit(dl)

INFO:root:Starting epoch 0/20
DEBUG:root:[1,     3] loss: 4.141e-01
DEBUG:root:[1,     6] loss: 1.236e-01


KeyboardInterrupt: 

In [11]:
tl = DataLoader(CaptchaDataset.from_dir(test_files), \
    4, # Fetch 4 samples per batch
    shuffle=True, num_workers=2)

net.validate(tl)

DEBUG:root:Predicted: aCCXw Ground Truth: IH379
DEBUG:root:Predicted: aSZXw Ground Truth: sjxMO
DEBUG:root:Predicted: kCCXX Ground Truth: BajPz
DEBUG:root:Predicted: kCCXX Ground Truth: cPRUK
DEBUG:root:Predicted: kS3lX Ground Truth: 5AQ4Q
DEBUG:root:Predicted: kS1lX Ground Truth: H4kPe
DEBUG:root:Predicted: kS3lX Ground Truth: iEyr8
DEBUG:root:Predicted: aSZXX Ground Truth: igWin
DEBUG:root:Predicted: 6S1lX Ground Truth: 279Wt
DEBUG:root:Predicted: ACCXX Ground Truth: HaYvg
DEBUG:root:Predicted: aSZXw Ground Truth: cwc7k
DEBUG:root:Predicted: aC3XX Ground Truth: GU3pg
DEBUG:root:Predicted: aC1XX Ground Truth: NWL25
DEBUG:root:Predicted: AC1XX Ground Truth: jkj7D
DEBUG:root:Predicted: kS3lX Ground Truth: fZXeV
DEBUG:root:Predicted: AACER Ground Truth: eesPt
DEBUG:root:Predicted: kS3lX Ground Truth: Okc3H
DEBUG:root:Predicted: 6cZXR Ground Truth: 2iPHn
DEBUG:root:Predicted: kS3lX Ground Truth: p9L3F
DEBUG:root:Predicted: aC3XX Ground Truth: XhV39
DEBUG:root:Predicted: aC3Xs Ground Truth

KeyboardInterrupt: 